In [ ]:
%%bash
# Install dependencies
%pip install numpy pandas matplotlib torch torchvision scikit-learn

   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   - -------------------------------------- 8.1/204.1 MB 50.1 MB/s eta 0:00:04
   --- ------------------------------------ 16.8/204.1 MB 44.0 MB/s eta 0:00:05
   ---- ----------------------------------- 23.9/204.1 MB 40.7 MB/s eta 0:00:05
   ------ --------------------------------- 30.7/204.1 MB 38.9 MB/s eta 0:00:05
   ------- -------------------------------- 38.0/204.1 MB 37.7 MB/s eta 0:00:05
   -------- ------------------------------- 45.4/204.1 MB 37.0 MB/s eta 0:00:05
   ---------- ----------------------------- 52.2/204.1 MB 36.5 MB/s eta 0:00:05
   ----------- ---------------------------- 59.5/204.1 MB 36.1 MB/s eta 0:00:05
   ------------- -------------------------- 66.8/204.1 MB 35.8 MB/s eta 0:00:04
   -------------- ------------------------- 73.7/204.1 MB 35.9 MB/s eta 0:00:04
   --------------- ------------------------ 80.5/204.1 MB 35.7 MB/s eta 0:00:04
   ----------------- ---------------------- 87.3/2

In [ ]:
# Import required libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image

import os
import cv2
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from image_dataloader import SocialSignalDataset
import torch
import torch.nn as nn

In [ ]:
# Create training and testing dataset
# Transform images

training_path = 'data/training'
testing_path = 'data/testing'

img_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset = SocialSignalDataset(root_dir=training_path, transform=img_transform)
# change batch size accordingly (my pc can only run up to 16)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True) 



In [ ]:
class SocialSignalModel(nn.Module):
    def __init__(self):
        super(SocialSignalModel, self).__init__()
        # TODO: IMPLEMENT
    def forward(self, x):
        # TODO: IMPLEMENT
        return x